# Introduction à la manipulation et à l'analyse d'images en Python

La manipulation et l'analise d'images en python est réaliser à l'aide du module [scikit-image](https://scikit-image.org/), qui un des nombreux projets de base de [scipy stack](https://www.scipy.org/docs.html)].  

Ce module est un dès plus utilisé pour la manipulation et l'analise d'image en python. La toute première version de scikit-image remonte à août 2009. La version actuelle, et celle utilisé dans cette démonstration de l'utilisation du module, est la version 0.18.3. Le dépot GitHub du projet scikit-image est consultable [ici](https://github.com/scikit-image).

Il est principalement utilisé pour réaliser de la segmentation, du filtrage, de la transformation que nous allons aborder au cours de cette introduction.


## contexte (william)




## objectifs (willliam)

L'objectif est d'explorer en surface le module scikit-image. Par exemple comment segmenter une image de microscopie multicellulaires pour analyser une cellule unique.
- identifier noyaux, organites, 
- filtrage objet abbérant
- mesure simple (taille, fluorescence, granulométrie)
- déterminer la corespondance cellulaire des fluorochromes

## ...


# « Images » Manipulation et analyse d'images en Python

Manipulation d’une image de microscopie avec scikit-image et analyses basiques.
  
## Questions et points à aborder ou à traiter :  

- Comment est codée informatiquement une image ?  (jeyani)
- Présenter les données : Quelles différences y a-t-il entre une image au format .tif et des formats d’images plus communs comme le .jpeg ? Quel intérêt d’utiliser le premier format en microscopie ?  (jeyani)
- Comment est composée l’image fournie dans comme donnée (différents canaux) ?  (tristan)
- Comment ouvrir et afficher l’image avec scikit-image ? (tristan)

Choisir un canal (par exemple le bleu) et extraire les principaux objets :  
    - Segmenter l’image (justifier le choix du seuil)
    - Identifier les objets et filtrer les objets aberrants  
    - Effectuer quelques mesures simples (taille, fluorescence  



### Comment est composée l'image fournie comme donnée ?

L'image FluorescentCells.tif est composée de trois canaux, un pour le rouge, un pour le vert, et un pour le bleu, les trois couleurs primaires. Cette image est codée dans un tableau numpy tridimensionnel : les première et deuxième dimensions correspondent aux coordonnées des pixels, la troisième dimension est composée des trois canaux rouge, vert, et bleu.

In [1]:
import skimage
import numpy
from matplotlib import pyplot

In [ ]:
import os

filename = filename = os.path.join("./FluorescentCells", "FluorescentCells.tif")
